In [1]:
import requests
import bs4
import dataclasses
import typing
import psycopg2
import os
import dataclasses

In [2]:
DB_URL=os.getenv("DB_URL")
connection = psycopg2.connect(
    DB_URL,
)

In [3]:
def get_urls():
    with connection.cursor() as cursor:
        cursor.execute(
            (
                "SELECT url"
                " FROM url;"
            ),
        )
        rows = cursor.fetchall()
    return [row[0] for row in rows]

In [4]:
urls = get_urls()

In [5]:
urls

['https://www.bjjheroes.com/?p=9246',
 'https://www.bjjheroes.com/?p=8494',
 'https://www.bjjheroes.com/?p=390',
 'https://www.bjjheroes.com/?p=1133',
 'https://www.bjjheroes.com/?p=13245',
 'https://www.bjjheroes.com/?p=7478',
 'https://www.bjjheroes.com/?p=6339',
 'https://www.bjjheroes.com/?p=8968',
 'https://www.bjjheroes.com/?p=9541',
 'https://www.bjjheroes.com/?p=13498',
 'https://www.bjjheroes.com/?p=699',
 'https://www.bjjheroes.com/?p=1116',
 'https://www.bjjheroes.com/?p=909',
 'https://www.bjjheroes.com/?p=4328',
 'https://www.bjjheroes.com/?p=2189',
 'https://www.bjjheroes.com/?p=289',
 'https://www.bjjheroes.com/?p=2583',
 'https://www.bjjheroes.com/?p=12235',
 'https://www.bjjheroes.com/?p=696',
 'https://www.bjjheroes.com/?p=2611',
 'https://www.bjjheroes.com/?p=9876',
 'https://www.bjjheroes.com/?p=7660',
 'https://www.bjjheroes.com/?p=11746',
 'https://www.bjjheroes.com/?p=13084',
 'https://www.bjjheroes.com/?p=1131',
 'https://www.bjjheroes.com/?p=1757',
 'https://ww

In [6]:
test_set = [
    "https://www.bjjheroes.com/?p=425",
    "https://www.bjjheroes.com/?p=144",
    "https://www.bjjheroes.com/?p=9246",
    "https://www.bjjheroes.com/?p=13245",
    "https://www.bjjheroes.com/?p=6550",
]

In [7]:
def insert_performance_rows(athlete_id, match_id, result):
    with connection.cursor() as cursor:
        cursor.execute(
            (
                "INSERT INTO performance (athlete_id, match_id, result)"
                " VALUES (%s, %s, %s)"
            ),
            (athlete_id, match_id, result),
        )
        connection.commit()

In [8]:
def create_athlete(name):
    with connection.cursor() as cursor:
        cursor.execute(
            (
                "INSERT INTO athlete (name)"
                " VALUES (%s)"
                " RETURNING id"
            ),
            (name,),
        )
        id_row = cursor.fetchone()
        id = id_row[0]
        connection.commit()
        return id

In [9]:
def get_athlete_by_url(url):
    with connection.cursor() as cursor:
        cursor.execute(
            (
                "SELECT a.id"
                " FROM athlete a"
                " JOIN url u ON u.athlete_id = a.id"
                " WHERE url = %s"
            ),
            (url,),
        )
        id_row = cursor.fetchone()
        if id_row:
            id = id_row[0]
            return id
        return None

In [10]:
def get_athlete_by_name(name):
    with connection.cursor() as cursor:
        cursor.execute(
            (
                "SELECT a.id"
                " FROM athlete a"
                " WHERE name = %s"
            ),
            (name,),
        )
        id_row = cursor.fetchone()
        if id_row:
            id = id_row[0]
            return id
        return None

In [11]:
def create_match(year, competition, method, stage, weight):
    with connection.cursor() as cursor:
        cursor.execute(
            (
                "INSERT INTO match (year, competition, method, stage, weight)"
                " VALUES (%s, %s, %s, %s, %s)"
                " RETURNING id"
            ),
            (year, competition, method, stage, weight),
        )
        id_row = cursor.fetchone()
        id = id_row[0]
        connection.commit()
        return id

In [12]:
def match_already_exists(year, competition, method, stage, weight, competitor_ids: list[int]):
    with connection.cursor() as cursor:
        cursor.execute(
            (
                "SELECT m.id"
                " FROM match m"
                " WHERE year = %s"
                " AND competition = %s"
                " AND method = %s"
                " AND stage = %s"
                " AND weight = %s"
            ),
            (year, competition, method, stage, weight,),
        )
        match_id_rows = cursor.fetchall()
        if match_id_rows:
            match_ids = [row[0] for row in match_id_rows]
            for match_id in match_ids:
                cursor.execute(
                    (
                        "SELECT a.id"
                        " FROM athlete a"
                        " JOIN performance p ON p.athlete_id = a.id"
                        " JOIN match m ON p.match_id = m.id"
                        " WHERE m.id = %s"
                    ),
                    (match_id,),
                )
                id_rows = cursor.fetchall()
                athlete_ids = {row[0] for row in id_rows}
                if competitor_ids[0] in athlete_ids and competitor_ids[1] in athlete_ids:
                    return match_id
        return False

In [13]:
test_set[0]

'https://www.bjjheroes.com/?p=425'

In [18]:
def scrape_matches(athlete_link):
    print(f"scraping match data for {athlete_link}")
    current_athlete_id = get_athlete_by_url(athlete_link)
    res = requests.get(athlete_link)
    bs = bs4.BeautifulSoup(res.content)
    table = bs.find("table", {"class": "table table-striped sort_table"})
    if table is None:
        return
    body = table.find("tbody")
    rows = body.find_all('tr')

    for row in rows:
        match_details = row.find_all("td")
        
        opponent = match_details[1]
        result = match_details[2].text
        method = match_details[3].text
        competition = match_details[4].text
        weight = match_details[5].text
        stage = match_details[6].text
        year = match_details[7].text
        
        opponent_name = opponent.text
        opponent_link = opponent.find('a')
        if opponent_link:
            opponent_link = f"https://www.bjjheroes.com{opponent_link.get('href')}"

        # check if the opponent exists in the database
        if opponent_link:
            opponent_id = get_athlete_by_url(opponent_link)
        else:
            opponent_id = get_athlete_by_name(opponent_name)
        if opponent_id is None:
            print(f"new athlete found: {opponent_name}")
            opponent_id = create_athlete(opponent_name)

        # check if match exists in database
        match_id = match_already_exists(
            year, 
            competition, 
            method, 
            stage, 
            weight, 
            [opponent_id, current_athlete_id]
        )
        if match_id:
            print(f"Match already found in database: {match_id}")
            continue
        
        match_id = create_match(year, competition, method, stage, weight)
        print(f"added match: {match_id}")
        possible_results = ['W', 'L', 'D']
        if result not in possible_results:
            raise ValueError(f"got result: {result}")
        if result == "D":
            opponent_result = "D"
        elif result == "W":
            opponent_result = "L"
        else:
            opponent_result = "W"
        insert_performance_rows(current_athlete_id, match_id, result)
        insert_performance_rows(opponent_id, match_id, opponent_result)

In [20]:
%%prun
for link in urls[0:10]:
    scrape_matches(link)

UsageError: Cell magic `%%memeit` not found.
